# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content. 

In [ ]:
!python -m pip install semantic-kernel==0.9.4b1

In [1]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

kernel = sk.Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat_gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

Let's define a prompt outlining a dialogue chat bot.

In [3]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [4]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.create_function_from_prompt(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [5]:
chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments

In [6]:
arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot

In [7]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Sure! I'd be happy to help. What genre or type of book are you interested in?


Update the history with the output

In [8]:
chat_history.add_assistant_message(str(response))

Keep Chatting!

In [9]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))

In [10]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Certainly! One fascinating book that delves into both history and philosophy is "The Greeks" by H. D. F. Kitto. This book provides a comprehensive and engaging exploration of ancient Greek civilization, covering topics such as Greek mythology, philosophy, politics, and society. It offers an insightful look into the influential ideas and events that shaped ancient Greece, making it a great choice for anyone interested in learning more about this captivating period in history.


In [11]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks" by H. D. F. Kitto offers a comprehensive exploration of ancient Greek civilization, covering various aspects of Greek life, including mythology, philosophy, politics, and society. The book provides insights into the influential ideas and events that shaped ancient Greece, making it a great choice for readers interested in delving into the rich history and philosophical contributions of this fascinating culture.


In [12]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: By reading "The Greeks" by H. D. F. Kitto, you will gain a deeper understanding of ancient Greek civilization, including its mythology, philosophy, politics, and society. The book delves into the origins of Greek civilization, the development of Greek city-states, the impact of Greek mythology and religion, the contributions of Greek philosophers, and the cultural and intellectual achievements of the ancient Greeks. You will also learn about key historical events and influential figures that shaped ancient Greece, providing a comprehensive overview of this captivating period in history.


In [13]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are a few more books about ancient Greece that you might find interesting:

1. "The Histories" by Herodotus - Considered one of the earliest works of history, this book provides a detailed account of the Greco-Persian Wars and offers valuable insights into ancient Greek culture and society.

2. "The Peloponnesian War" by Thucydides - This classic work offers a thorough and analytical account of the Peloponnesian War, providing a unique perspective on ancient Greek military and political history.

3. "The Iliad" and "The Odyssey" by Homer - These epic poems are essential reading for anyone interested in ancient Greek literature and mythology, offering timeless stories of heroism, adventure, and the Greek gods.

4. "The Republic" by Plato - This influential philosophical work explores the nature of justice, the ideal society, and the role of the philosopher-king, offering valuable insights into a

In [14]:
await chat("what are some important themes from The Iliad?")

User: what are some important themes from The Iliad?
ChatBot: Some important themes from "The Iliad" include:

1. War and its consequences: The epic poem explores the devastating impact of war on individuals, families, and society, depicting the brutality and tragedy of battle.

2. Honor and glory: The concept of honor and the pursuit of glory are central themes in "The Iliad," as warriors seek to earn renown through heroic deeds on the battlefield.

3. Fate and free will: The tension between fate and free will is a recurring theme, as characters grapple with their predetermined destinies while also making choices that shape their actions and outcomes.

4. Gods and mortals: The intervention of the Greek gods in human affairs plays a significant role in the narrative, highlighting the complex relationship between divine influence and mortal agency.

5. Friendship and loyalty: The bonds of friendship and loyalty are explored through the relationships between characters, emphasizing the i

In [15]:
await chat("what are some interesting aspects about the role of women in Plato's The Republic?")

User: what are some interesting aspects about the role of women in Plato's The Republic?
ChatBot: In "The Republic," Plato presents an intriguing perspective on the role of women in society. He challenges traditional gender roles and explores the potential equality of women in areas such as education, governance, and the guardianship of the ideal state. Some interesting aspects about the role of women in "The Republic" include:

1. Equality in education: Plato advocates for equal education for men and women in the ideal city, emphasizing the importance of cultivating intellectual and moral virtues in both genders.

2. Women in the guardian class: Plato suggests that women should have the same opportunities as men to become guardians of the state, emphasizing their potential for leadership and the defense of the city.

3. Community of women and children: In the ideal city, Plato proposes the concept of a "community of women and children," challenging traditional notions of family and pr

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [16]:
print(chat_history)

<chat_history><message role="system">You are a helpful chatbot who is good about giving book recommendations.</message><message role="assistant">Sure! I'd be happy to help. What genre or type of book are you interested in?</message><message role="user">I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</message><message role="assistant">Certainly! One fascinating book that delves into both history and philosophy is "The Greeks" by H. D. F. Kitto. This book provides a comprehensive and engaging exploration of ancient Greek civilization, covering topics such as Greek mythology, philosophy, politics, and society. It offers an insightful look into the influential ideas and events that shaped ancient Greece, making it a great choice for anyone interested in learning more about this captivating period in history.</message><message role="user">that sounds interesting, what is it about?</message><message role="assistant">"The Greeks" by H. D. F. Kitto 